In [26]:
!pip install tensorflow-hub
!pip install tensorflow-datasets
!pip install tf-keras

In [51]:
import os
import numpy as np
# tensorflow packages 
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tf_keras as keras

print("version:", tf.__version__)
print("Eager mode:", tf.executing_eagerly())
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

version: 2.16.2
Eager mode: True
Hub version: 0.16.1
GPU is NOT AVAILABLE


In [28]:
train_data, validation_data, test_data = tfds.load(name="imdb_reviews",
                                                  split=('train[:60%]', 'train[60%:]', 'test'),
                                                  as_supervised=True)

In [29]:
print(tf.data.experimental.cardinality(train_data))
print(tf.data.experimental.cardinality(validation_data))
print(tf.data.experimental.cardinality(test_data))

tf.Tensor(15000, shape=(), dtype=int64)
tf.Tensor(10000, shape=(), dtype=int64)
tf.Tensor(25000, shape=(), dtype=int64)


In [30]:
train_example_batch, train_labels_batch = next(iter(train_data.batch(10)))

In [31]:
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [32]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

In [52]:
embed = hub.load("https://www.kaggle.com/models/google/nnlm/TensorFlow2/en-dim50/1")
embeddings = embed(["cat is on the mat", "dog is in the fog"])
hub_layer = hub.KerasLayer(embedding,
                          input_shape=[],
                          dtype = tf.string,
                          trainable = True
                          )

In [53]:
hub_layer(train_example_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [54]:
model = keras.Sequential()
model.add(hub_layer)
model.add(keras.layers.Dense(16, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_8 (KerasLayer)  (None, 50)                48190600  
                                                                 
 dense_8 (Dense)             (None, 16)                816       
                                                                 
 dense_9 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48191433 (183.84 MB)
Trainable params: 48191433 (183.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [45]:
model.compile(optimizer='adam',
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [48]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 3s 72ms/step - loss: 0.5130 - accuracy: 0.7495 - val_loss: 0.5136 - val_accuracy: 0.7523
Epoch 2/10
30/30 [==============================] - 3s 92ms/step - loss: 0.5117 - accuracy: 0.7485 - val_loss: 0.5131 - val_accuracy: 0.7537
Epoch 3/10
30/30 [==============================] - 3s 87ms/step - loss: 0.5110 - accuracy: 0.7487 - val_loss: 0.5130 - val_accuracy: 0.7534
Epoch 4/10
30/30 [==============================] - 2s 66ms/step - loss: 0.5105 - accuracy: 0.7496 - val_loss: 0.5128 - val_accuracy: 0.7547
Epoch 5/10
30/30 [==============================] - 2s 61ms/step - loss: 0.5102 - accuracy: 0.7475 - val_loss: 0.5127 - val_accuracy: 0.7551
Epoch 6/10
30/30 [==============================] - 2s 61ms/step - loss: 0.5099 - accuracy: 0.7511 - val_loss: 0.5120 - val_accuracy: 0.7549
Epoch 7/10
30/30 [==============================] - 2s 65ms/step - loss: 0.5095 - accuracy: 0.7508 - val_loss: 0.5118 - val_accuracy: 0.7541
Epoch 8/10
30

In [50]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.5198 - accuracy: 0.7412 - 2s/epoch - 44ms/step
loss: 0.520
accuracy: 0.741
